In [ ]:
import nbformat as nbf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from google.colab import files
import io

In [ ]:
uploaded = files.upload()

Saving dataset_Facebook - dataset_Facebook.csv.csv to dataset_Facebook - dataset_Facebook.csv.csv


In [ ]:
data_original = pd.read_csv(io.BytesIO(uploaded['dataset_Facebook - dataset_Facebook.csv.csv']))
data_original.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumption,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [ ]:
#Extract Data
selected_columns = ['Post Hour', 'Post Weekday', 'Lifetime Post Total Reach']
data = data_original[selected_columns]
df = pd.DataFrame(data)
df

,Post Hour,Post Weekday,Lifetime Post Total Reach
0,3,4,2752
1,10,3,10460
2,3,3,2413
3,10,2,50128
4,3,2,7244
...,...,...,...
495,2,7,4684
496,8,5,3480
497,2,5,3778
498,11,4,4156


In [ ]:
dependent_variable = df['Lifetime Post Total Reach']
group = df['Post Weekday']
covariate = df['Post Hour']

In [ ]:
# ANCOVA
model_ancova = ols('dependent_variable ~ group + covariate', data=df).fit()
ancova_results = sm.stats.anova_lm(model_ancova, typ=2)

# Display the ANCOVA table
print(ancova_results)

# Tukey's Post Hoc Test using ANCOVA
# We will use the model's residuals as the dependent variable
residuals = model_ancova.resid
tukey_ancova = pairwise_tukeyhsd(endog=residuals, groups=df['Post Weekday'], alpha=0.05)

# Display Tukey's test results after ANCOVA
print(tukey_ancova)


                 sum_sq     df         F    PR(>F)
group      6.544840e+08    1.0  1.263722  0.261490
covariate  8.218885e+06    1.0  0.015870  0.899803
Residual   2.573972e+11  497.0       NaN       NaN
     Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj     lower      upper    reject
-------------------------------------------------------------
     1      2  5057.8282 0.8585  -6589.9979 16705.6543  False
     1      3  6232.7757 0.7005  -5507.0439 17972.5953  False
     1      4  4960.4068 0.8572  -6438.4741 16359.2877  False
     1      5  3068.9579 0.9865  -8534.6736 14672.5894  False
     1      6   3814.015 0.9497  -7273.0112 14901.0411  False
     1      7  2069.7613 0.9979   -8986.369 13125.8916  False
     2      3  1174.9475 0.9999 -10650.8006 13000.6956  False
     2      4   -97.4214    1.0 -11584.7814 11389.9385  False
     2      5 -1988.8703 0.9988 -13679.4313  9701.6907  False
     2      6 -1243.8132 0.9999 -12421.7872  9934.16